In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import recall_score, precision_score, f1_score

### Load the Data

In [2]:
data = pd.read_excel('Concrete_Data.xls')

### Splitting the Data
- 80% -> Train
- 20% -> Validation

In [3]:
nrow = data.shape[0]

In [4]:
train = data.loc[:nrow * 0.8]
test = data.loc[nrow * 0.8:]

In [5]:
X_train = np.array(train.iloc[:, :-1])
y_train = np.array(train['Concrete compressive strength(MPa, megapascals) '])

X_val = np.array(test.iloc[:, :-1])
y_val = np.array(test['Concrete compressive strength(MPa, megapascals) '])

In [6]:
X_train.shape, X_val.shape

((825, 8), (206, 8))

In [7]:
l = len(y_train)

### Custom Function for searching theta, cost

In [8]:
def normalize(x):
    mu = np.mean(x, axis = 0)
    sig = np.std(x, axis= 0, ddof = 1)
    norm = (x - mu)/sig
    return norm, mu, sig

In [9]:
def cost_search(X, y, theta):
    predictions = X.dot(theta)
    errors = np.subtract(predictions, y)
    sqrErrors = np.square(errors)
    return 1/(2 * l) * errors.T.dot(errors)

In [10]:
def gradient(X, y, theta, learning_rate, iterations):
    cost = np.zeros(iterations)
    for i in range(iterations):
        prd = X.dot(theta)
        error = np.subtract(prd, y)
        delta = (learning_rate / l) * X.transpose().dot(error);
        theta = theta - delta;
        cost[i] = cost_search(X, y, theta)
    return theta, cost

### Get the model

In [11]:
X, mu, sig = normalize(X_train)
X = np.hstack((np.ones((l, 1)), X))
theta = np.zeros(X_train.shape[1] + 1)
iterations = 500
learning_rate = 0.15
theta, cost = gradient(X, y_train, theta, learning_rate, iterations)

### Prediction

In [12]:
test_data = ((X_val - mu) / sig)

In [13]:
target_values = []

In [14]:
for n in range(test_data.shape[0]) :
    target = np.hstack((np.ones(1), test_data[n])).dot(theta)
    target_values.append(target)

### Evaluation - MAE(Mean Absolute Error)

In [15]:
result = pd.DataFrame({'Actual' : y_val, 'Prediction' : target_values})

In [16]:
result.head()

,Actual,Prediction
0,21.752968,18.827615
1,39.093289,32.603310
2,24.393661,35.609625
3,50.511012,48.371938
4,74.987410,50.800604


In [17]:
mae = np.sum(abs(y_val - target_values)) / len(y_val)

In [18]:
mae

6.927617208915032

***
## Classification

In [19]:
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))

활성화 함수인 Softmax 구현

In [20]:
def get_class(x) :
    if x <= 21 :
        return 1
    elif 22 <= x < 45 :
        return 2
    else :
        return 3

MPa기준으로 그룹핑 함수 구현

In [21]:
def affine_layer_forward(X, W, b):
    y = X.dot(W) + b
    cache = (X, W, b)
    return y, cache

affine 층 구현

In [22]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x)) 

시그모이드 함수 구현

In [23]:
data['target'] = data['Concrete compressive strength(MPa, megapascals) '].apply(get_class)

In [24]:
X_train = pd.DataFrame()
X_val = pd.DataFrame()
X_test = pd.DataFrame()

In [25]:
for i in [1,2,3] :
    df = data[data.target == i]
    tr = df.sample(int(df.shape[0] * 0.7))
    val = df.query('index not in @tr.index')
    val = val.sample(int(val.shape[0] * (2/3)))
    te = df.query('index not in @tr.index and index not in @val.index')
    
    X_train = pd.concat([X_train, tr], axis = 0, ignore_index = True)
    X_val = pd.concat([X_val, val], axis = 0, ignore_index = True)
    X_test = pd.concat([X_test, te], axis = 0, ignore_index = True)

In [26]:
y_train = pd.get_dummies(X_train.iloc[:, -1] - 1).to_numpy()
y_val = pd.get_dummies(X_val.iloc[:, -1] - 1).to_numpy()
y_test = pd.get_dummies(X_test.iloc[:, -1] - 1).to_numpy()
X_train = X_train.iloc[:, :-2].to_numpy()
X_val = X_val.iloc[:, :-2].to_numpy()
X_test = X_test.iloc[:, :-2].to_numpy()

In [27]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size

In [28]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

In [29]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    return W1, b1, W2, b2

In [30]:
def affine_layer_backward(dy, cache):
    X, W, b = cache
    dX = np.dot(dy, W.T)
    dW = np.dot(X.T, dy)
    db = np.sum(dy, axis=0)
    return dX, dW, db

In [31]:
def train_step(X, Y, W1, b1, W2, b2, learning_rate, verbose=False):
    a1, cache1 = affine_layer_forward(X, W1, b1)
    z1 = sigmoid(a1)
    a2, cache2 = affine_layer_forward(z1, W2, b2)
    y_hat = softmax(a2)
    Loss = cross_entropy_error(y_hat, Y)
  
    dy = (y_hat - Y) / X.shape[0]
    dz1, dW2, db2 = affine_layer_backward(dy, cache2)
    da1 = sigmoid_grad(a1) * dz1
    dX, dW1, db1 = affine_layer_backward(da1, cache1)
    
    W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)
    
    return W1, b1, W2, b2, Loss

In [32]:
def predict(W1, b1, W2, b2, X):
    a1 = X.dot(W1) + b1
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2)

    return y

In [33]:
def init_params(input_size, hidden_size, output_size, weight_init_std=0.1):

    W1 = weight_init_std * np.random.randn(input_size, hidden_size)
    b1 = np.zeros(hidden_size)
    W2 = weight_init_std * np.random.randn(hidden_size, output_size)
    b2 = np.zeros(output_size)
    
    return W1, b1, W2, b2

In [34]:
iters_num = 100000  # 반복 횟수를 적절히 설정한다.
train_size = X_train.shape[0]
batch_size = 32   # 미니배치 크기
learning_rate = 0.1

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

W1, b1, W2, b2 = init_params(8, 32, 3)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]
    
    W1, b1, W2, b2, Loss = train_step(x_batch, y_batch, W1, b1, W2, b2, learning_rate=0.25, verbose=False)
    
    if i % iter_per_epoch == 0:
        train_pred = np.argmax(predict(W1, b1, W2, b2, X_train), axis = 1)
        val_pred = np.argmax(predict(W1, b1, W2, b2, X_val), axis = 1)
        test_pred = np.argmax(predict(W1, b1, W2, b2, X_test), axis = 1)

tr_r_score = recall_score(np.argmax(y_train, axis = 1), train_pred, average = 'weighted')
val_r_score = recall_score(np.argmax(y_val, axis = 1), val_pred, average = 'weighted')
te_r_score = recall_score(np.argmax(y_test, axis = 1), test_pred, average = 'weighted')

tr_p_score = precision_score(np.argmax(y_train, axis = 1), train_pred, average = 'weighted')
val_p_score = precision_score(np.argmax(y_val, axis = 1), val_pred, average = 'weighted')
te_p_score = precision_score(np.argmax(y_test, axis = 1), test_pred, average = 'weighted')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
print('Recall Score')
print(f'Train | {tr_r_score}')
print(f'Validation | {val_r_score}')
print(f'Test | {te_r_score}')

print('\nPrecision Score')
print(f'Train | {tr_p_score}')
print(f'Validation | {val_p_score}')
print(f'Test | {te_p_score}')

Recall Score
Train | 0.5159944367176634
Validation | 0.5145631067961165
Test | 0.5142857142857142

Precision Score
Train | 0.26625025872357877
Validation | 0.2647751908756716
Test | 0.2644897959183673


***

In [36]:
from scipy.io import loadmat
# Use loadmat to load matlab files
mat=loadmat("ex3data1.mat")
# mat is a dict with key "X" for x-values, and key "y" for y values
X = mat["X"]
y = [v[0] for v in mat["y"]]

In [37]:
y = pd.get_dummies(y).to_numpy()

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42, stratify = y)

In [40]:
seed = 884736743
rng = np.random.default_rng(seed)

In [41]:
def relu (x):
    return (x>=0) * x

# 양의 입력에 대해 1을, 그렇지 않으면 0을 반환하는 ReLU 기능의 파생 모델을 설정
def relu2deriv(output):
    return output >= 0

In [42]:
def NN(learning_rate, epochs, hidden_size, n_features, num_labels) :

    weights_1 = 0.2 * rng.random((n_features, hidden_size)) - 0.1
    weights_2 = 0.2 * rng.random((hidden_size, num_labels)) - 0.1

    store_training_loss = []
    store_test_loss = []

    # 훈련 루프 시작
    # 정의된 에포크(반복)의 수에 대한 학습 실험을 실행
    for j in range(epochs):

        #################
        #    훈련 단계   #
        #################


        # 훈련셋의 모든 데이터에 대한 순전파와 역전파를 수행하고 그에 따른 가중치 조정
        for i in range(len(X_train)):
            layer_0 = X_train[i]
            layer_1 = np.dot(layer_0, weights_1)
            layer_1 = relu(layer_1)
            dropout_mask = rng.integers(low=0, high=2, size=layer_1.shape)
            layer_1 *= dropout_mask * 2
            layer_2 = np.dot(layer_1, weights_2)

            # 역전파:

            layer_2_delta = (y_train[i] - layer_2)
            layer_1_delta = np.dot(weights_2, layer_2_delta) * relu2deriv(layer_1)
            layer_1_delta *= dropout_mask
            weights_1 += learning_rate * np.outer(layer_0, layer_1_delta)
            weights_2 += learning_rate * np.outer(layer_1, layer_2_delta)


        ###################
        #    측정 단계     #
        ###################

        train_pred = relu(X_train @ weights_1) @ weights_2
        test_pred = relu(X_test @ weights_1) @ weights_2


        # 세트별 성능 측정
        tr_recall = round(recall_score(np.argmax(y_train, axis = 1), np.argmax(train_pred, axis = 1), average = 'weighted'), 3)
        te_recall = round(recall_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1), average = 'weighted'), 3)

        tr_precision = round(precision_score(np.argmax(y_train, axis = 1), np.argmax(train_pred, axis = 1), average = 'weighted'), 3)
        te_precision = round(precision_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1), average = 'weighted'), 3)

        tr_f1 = round(f1_score(np.argmax(y_train, axis = 1), np.argmax(train_pred, axis = 1), average = 'weighted'), 3)
        te_f1 = round(f1_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1), average = 'weighted'), 3)


        # 각 에포크에서 성능 출력
        print(f'########## Epoch {j + 1} #########')
        print(f'Summary of Train Set')
        print(f'Recall : {tr_recall} / Precision : {tr_precision} / F1 : {tr_f1}')
        print(f'\nSummary of Test Set')
        print(f'Recall : {te_recall} / Precision : {te_precision} / F1 : {te_f1}\n')

함수 구성방식(학습률, 에포크, 은닉층 노드수, 인풋데이터 컬럼 수, 레이블 수)

In [43]:
NN(0.003, 20, 5, 400, 10)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


########## Epoch 1 #########
Summary of Train Set
Recall : 0.315 / Precision : 0.213 / F1 : 0.229

Summary of Test Set
Recall : 0.314 / Precision : 0.209 / F1 : 0.231

########## Epoch 2 #########
Summary of Train Set
Recall : 0.286 / Precision : 0.235 / F1 : 0.192

Summary of Test Set
Recall : 0.28 / Precision : 0.213 / F1 : 0.193

########## Epoch 3 #########
Summary of Train Set
Recall : 0.342 / Precision : 0.259 / F1 : 0.225

Summary of Test Set
Recall : 0.334 / Precision : 0.257 / F1 : 0.215

########## Epoch 4 #########
Summary of Train Set
Recall : 0.386 / Precision : 0.234 / F1 : 0.255

Summary of Test Set
Recall : 0.38 / Precision : 0.232 / F1 : 0.25

########## Epoch 5 #########
Summary of Train Set
Recall : 0.424 / Precision : 0.323 / F1 : 0.33

Summary of Test Set
Recall : 0.406 / Precision : 0.307 / F1 : 0.308

########## Epoch 6 #########
Summary of Train Set
Recall : 0.388 / Precision : 0.234 / F1 : 0.261

Summary of Test Set
Recall : 0.376 / Precision : 0.221 / F1 : 0.2

In [44]:
NN(0.003, 20, 10, 400, 10)

########## Epoch 1 #########
Summary of Train Set
Recall : 0.526 / Precision : 0.563 / F1 : 0.47

Summary of Test Set
Recall : 0.508 / Precision : 0.531 / F1 : 0.45

########## Epoch 2 #########
Summary of Train Set
Recall : 0.562 / Precision : 0.587 / F1 : 0.511

Summary of Test Set
Recall : 0.568 / Precision : 0.547 / F1 : 0.524

########## Epoch 3 #########
Summary of Train Set
Recall : 0.538 / Precision : 0.581 / F1 : 0.465

Summary of Test Set
Recall : 0.552 / Precision : 0.674 / F1 : 0.486

########## Epoch 4 #########
Summary of Train Set
Recall : 0.559 / Precision : 0.63 / F1 : 0.496

Summary of Test Set
Recall : 0.57 / Precision : 0.615 / F1 : 0.51

########## Epoch 5 #########
Summary of Train Set
Recall : 0.573 / Precision : 0.675 / F1 : 0.514

Summary of Test Set
Recall : 0.58 / Precision : 0.607 / F1 : 0.52

########## Epoch 6 #########
Summary of Train Set
Recall : 0.608 / Precision : 0.594 / F1 : 0.553

Summary of Test Set
Recall : 0.594 / Precision : 0.721 / F1 : 0.543


In [45]:
NN(0.003, 20, 30, 400, 10)

########## Epoch 1 #########
Summary of Train Set
Recall : 0.63 / Precision : 0.601 / F1 : 0.585

Summary of Test Set
Recall : 0.604 / Precision : 0.576 / F1 : 0.555

########## Epoch 2 #########
Summary of Train Set
Recall : 0.726 / Precision : 0.758 / F1 : 0.688

Summary of Test Set
Recall : 0.706 / Precision : 0.664 / F1 : 0.669

########## Epoch 3 #########
Summary of Train Set
Recall : 0.753 / Precision : 0.783 / F1 : 0.728

Summary of Test Set
Recall : 0.734 / Precision : 0.769 / F1 : 0.712

########## Epoch 4 #########
Summary of Train Set
Recall : 0.787 / Precision : 0.793 / F1 : 0.78

Summary of Test Set
Recall : 0.776 / Precision : 0.785 / F1 : 0.772

########## Epoch 5 #########
Summary of Train Set
Recall : 0.794 / Precision : 0.802 / F1 : 0.784

Summary of Test Set
Recall : 0.782 / Precision : 0.792 / F1 : 0.772

########## Epoch 6 #########
Summary of Train Set
Recall : 0.8 / Precision : 0.803 / F1 : 0.797

Summary of Test Set
Recall : 0.808 / Precision : 0.815 / F1 : 0.8

In [46]:
NN(0.003, 20, 50, 400, 10)

########## Epoch 1 #########
Summary of Train Set
Recall : 0.751 / Precision : 0.783 / F1 : 0.735

Summary of Test Set
Recall : 0.752 / Precision : 0.777 / F1 : 0.732

########## Epoch 2 #########
Summary of Train Set
Recall : 0.787 / Precision : 0.806 / F1 : 0.777

Summary of Test Set
Recall : 0.774 / Precision : 0.794 / F1 : 0.764

########## Epoch 3 #########
Summary of Train Set
Recall : 0.802 / Precision : 0.82 / F1 : 0.796

Summary of Test Set
Recall : 0.774 / Precision : 0.797 / F1 : 0.767

########## Epoch 4 #########
Summary of Train Set
Recall : 0.817 / Precision : 0.832 / F1 : 0.813

Summary of Test Set
Recall : 0.782 / Precision : 0.806 / F1 : 0.778

########## Epoch 5 #########
Summary of Train Set
Recall : 0.819 / Precision : 0.833 / F1 : 0.818

Summary of Test Set
Recall : 0.796 / Precision : 0.817 / F1 : 0.795

########## Epoch 6 #########
Summary of Train Set
Recall : 0.833 / Precision : 0.846 / F1 : 0.833

Summary of Test Set
Recall : 0.798 / Precision : 0.826 / F1 : 

In [47]:
def NN(learning_rate, epochs, hidden_size1, hidden_size2, n_features, num_labels) :

    weights_1 = 0.2 * rng.random((n_features, hidden_size1)) - 0.1
    weights_2 = 0.2 * rng.random((hidden_size1, hidden_size2)) - 0.1
    weights_3 = 0.2 * rng.random((hidden_size2, num_labels)) - 0.1
    
    store_training_loss = []
    store_test_loss = []

    # 훈련 루프 시작
    # 정의된 에포크(반복)의 수에 대한 학습 실험을 실행
    for j in range(epochs):

        #################
        #    훈련 단계   #
        #################

        # 훈련셋의 모든 데이터에 대한 순전파와 역전파를 수행하고 그에 따른 가중치 조정
        for i in range(len(X_train)):
            layer_0 = X_train[i]
            layer_1 = np.dot(layer_0, weights_1)
            layer_1 = relu(layer_1)
            layer_2 = np.dot(layer_1, weights_2)
            layer_2 = relu(layer_2)
            dropout_mask = rng.integers(low=0, high=2, size=layer_2.shape)
            layer_2 *= dropout_mask * 2
            layer_3 = np.dot(layer_2, weights_3)
            
            # 역전파:
        
            layer_3_delta = (y_train[i] - layer_3)
            layer_2_delta = np.dot(weights_3, layer_3_delta) * relu2deriv(layer_2)
            layer_2_delta *= dropout_mask
            layer_1_delta = np.dot(weights_2, layer_2_delta) * relu2deriv(layer_1)
                                 
            weights_1 += learning_rate * np.outer(layer_0, layer_1_delta)
            weights_2 += learning_rate * np.outer(layer_1, layer_2_delta)
            weights_3 += learning_rate * np.outer(layer_2, layer_3_delta)


        ###################
        #    측정 단계     #
        ###################

        train_pred = relu(relu(X_train @ weights_1) @ weights_2) @ weights_3
        test_pred = relu(relu(X_test @ weights_1) @ weights_2) @ weights_3


        # 세트별 성능측정
        tr_recall = round(recall_score(np.argmax(y_train, axis = 1), np.argmax(train_pred, axis = 1), average = 'weighted'), 3)
        te_recall = round(recall_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1), average = 'weighted'), 3)

        tr_precision = round(precision_score(np.argmax(y_train, axis = 1), np.argmax(train_pred, axis = 1), average = 'weighted'), 3)
        te_precision = round(precision_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1), average = 'weighted'), 3)

        tr_f1 = round(f1_score(np.argmax(y_train, axis = 1), np.argmax(train_pred, axis = 1), average = 'weighted'), 3)
        te_f1 = round(f1_score(np.argmax(y_test, axis = 1), np.argmax(test_pred, axis = 1), average = 'weighted'), 3)

        # 각 에포크에서 성능 출력
        print(f'########## Epoch {j + 1} #########')
        print(f'Summary of Train Set')
        print(f'Recall : {tr_recall} / Precision : {tr_precision} / F1 : {tr_f1}')
        print(f'\nSummary of Test Set')
        print(f'Recall : {te_recall} / Precision : {te_precision} / F1 : {te_f1}\n')

In [48]:
NN(0.003, 20, 50, 10, 400, 10)

########## Epoch 1 #########
Summary of Train Set
Recall : 0.353 / Precision : 0.337 / F1 : 0.299

Summary of Test Set
Recall : 0.334 / Precision : 0.31 / F1 : 0.279

########## Epoch 2 #########
Summary of Train Set
Recall : 0.544 / Precision : 0.448 / F1 : 0.483

Summary of Test Set
Recall : 0.544 / Precision : 0.454 / F1 : 0.485

########## Epoch 3 #########
Summary of Train Set
Recall : 0.524 / Precision : 0.554 / F1 : 0.435

Summary of Test Set
Recall : 0.498 / Precision : 0.397 / F1 : 0.407

########## Epoch 4 #########
Summary of Train Set
Recall : 0.54 / Precision : 0.53 / F1 : 0.453

Summary of Test Set
Recall : 0.512 / Precision : 0.496 / F1 : 0.432

########## Epoch 5 #########
Summary of Train Set
Recall : 0.525 / Precision : 0.458 / F1 : 0.46

Summary of Test Set
Recall : 0.502 / Precision : 0.434 / F1 : 0.44

########## Epoch 6 #########
Summary of Train Set
Recall : 0.555 / Precision : 0.468 / F1 : 0.486

Summary of Test Set
Recall : 0.542 / Precision : 0.465 / F1 : 0.47

<ipython-input-41-a4b059e7c946>:2: RuntimeWarning: invalid value encountered in multiply
  return (x>=0) * x


########## Epoch 14 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 15 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 16 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 17 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 18 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 19 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 20 ##

In [49]:
NN(0.003, 20, 40, 20, 400, 10)

########## Epoch 1 #########
Summary of Train Set
Recall : 0.444 / Precision : 0.554 / F1 : 0.371

Summary of Test Set
Recall : 0.44 / Precision : 0.347 / F1 : 0.36

########## Epoch 2 #########
Summary of Train Set
Recall : 0.59 / Precision : 0.61 / F1 : 0.544

Summary of Test Set
Recall : 0.596 / Precision : 0.609 / F1 : 0.544

########## Epoch 3 #########
Summary of Train Set
Recall : 0.616 / Precision : 0.608 / F1 : 0.569

Summary of Test Set
Recall : 0.642 / Precision : 0.629 / F1 : 0.594

########## Epoch 4 #########
Summary of Train Set
Recall : 0.633 / Precision : 0.627 / F1 : 0.585

Summary of Test Set
Recall : 0.644 / Precision : 0.639 / F1 : 0.596

########## Epoch 5 #########
Summary of Train Set
Recall : 0.604 / Precision : 0.645 / F1 : 0.552

Summary of Test Set
Recall : 0.572 / Precision : 0.515 / F1 : 0.521

########## Epoch 6 #########
Summary of Train Set
Recall : 0.513 / Precision : 0.634 / F1 : 0.487

Summary of Test Set
Recall : 0.492 / Precision : 0.627 / F1 : 0.4

<ipython-input-41-a4b059e7c946>:2: RuntimeWarning: invalid value encountered in multiply
  return (x>=0) * x
<ipython-input-47-fa371d45b95e>:26: RuntimeWarning: invalid value encountered in multiply
  layer_2 *= dropout_mask * 2


########## Epoch 12 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 13 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 14 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 15 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 16 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 17 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 18 ##

In [50]:
NN(0.003, 20, 50, 10, 400, 10)

########## Epoch 1 #########
Summary of Train Set
Recall : 0.305 / Precision : 0.267 / F1 : 0.218

Summary of Test Set
Recall : 0.322 / Precision : 0.268 / F1 : 0.229

########## Epoch 2 #########
Summary of Train Set
Recall : 0.507 / Precision : 0.433 / F1 : 0.431

Summary of Test Set
Recall : 0.526 / Precision : 0.403 / F1 : 0.443

########## Epoch 3 #########
Summary of Train Set
Recall : 0.502 / Precision : 0.448 / F1 : 0.405

Summary of Test Set
Recall : 0.516 / Precision : 0.471 / F1 : 0.415

########## Epoch 4 #########
Summary of Train Set
Recall : 0.532 / Precision : 0.473 / F1 : 0.445

Summary of Test Set
Recall : 0.536 / Precision : 0.482 / F1 : 0.45

########## Epoch 5 #########
Summary of Train Set
Recall : 0.519 / Precision : 0.486 / F1 : 0.43

Summary of Test Set
Recall : 0.5 / Precision : 0.458 / F1 : 0.408

########## Epoch 6 #########
Summary of Train Set
Recall : 0.51 / Precision : 0.47 / F1 : 0.41

Summary of Test Set
Recall : 0.502 / Precision : 0.508 / F1 : 0.398


<ipython-input-47-fa371d45b95e>:33: RuntimeWarning: invalid value encountered in multiply
  layer_2_delta *= dropout_mask
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/numeric.py:909: RuntimeWarning: invalid value encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)


########## Epoch 16 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 17 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 18 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 19 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

########## Epoch 20 #########
Summary of Train Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

Summary of Test Set
Recall : 0.1 / Precision : 0.01 / F1 : 0.018

